In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
test_data = {}
# blocks 1 and 3 are gain and have 4 for win and 3 for loss
# blocks 2 and 4 are loss and have 2 for win and 1 for loss
for i in range(1,101):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    block = [
                {
                    'action': np.array([[resp-1 for ([id],[resp]) in zip(struct[0,0]['blockID'],struct[0,0]['resp']) if id==blockID]]),
                    'state': np.array([[stimOffers for ([id],stimOffers) in zip(struct[0,0]['blockID'],struct[0,0]['stimOffers']) if id==blockID]]),
                    'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew']) if id==blockID]]),
                    'id': 'S'+str(i),

                    'block': blockID
                } for blockID in np.unique([struct[0,0]['blockID']])

            ]
#     block = [
#                 {
#                     'action': np.array([struct[0,0]['resp'].flatten()])-1,
#                     'state': np.array([struct[0,0]['stimOffers']]),
#                     'reward': np.array([[id%2+int(rew/25)+2 for ([id],[rew]) in zip(struct[0,0]['blockID'],struct[0,0]['outcomeRew'])]]),
#                     'id': 'S'+str(i),

#                     'block': 0
#                 }

#             ]
    if i<91:
        data['S'+str(i)] = block
    else:
        test_data['S'+str(i)] = block

In [3]:
data

{'S1': [{'action': array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
           1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
           1, 0, 1, 0]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],
           [1, 2],
           [2, 1],
           [3, 2],
           [3, 2],
           [2, 3],
           [3, 1],
           [2, 3],
           [1, 3],
           [3, 2],
           [2, 1],
           [2, 1],
           [3, 2],
           [1, 3],
           [2, 1],
           [2, 1],
           [2, 3],
           [1, 2],
           [2, 3],
           [3, 2],
           [1, 3],
           [3, 1],
           [3, 2],
           [3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [2, 3

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=20)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/simpleModel_rewardOutcome/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-07-02 14:25:42,652 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:25:42,654 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:25:42,655 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:25:42,657 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:25:42,658 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-07-02 14:25:47,107 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/simpleModel_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:26:03,237 - DeepRL - DEBUG - global iter =    0 total obj: 11927.9060
2021-07-02 14:26:16,439 - DeepRL - DEBUG - global iter =    1 total obj: 11734.8142
2021-07-02 14:26:32,734 - DeepRL - DEBUG - global iter =    2 total obj: 11344.8868
2021-07-02 14:26:48,483 - DeepRL - DEBUG - global iter =    3 total obj: 10588.0876
2021-07-02 14:27:04,751 - DeepRL - DEBUG - global iter =    4 total obj: 9790.2490
2021-07-02 14:27:20,677 - DeepRL - DEBUG - global iter =    5 total obj: 9195.1554
2021-07-02 14:27:36,253 - DeepRL - DEBUG - global iter =    6 total obj: 8811.8348
2021-07-02 14:27:51,357 - DeepRL - DEBUG - global iter =    7 total obj: 8321.1410
2021-07-02 14:28:07,949 - DeepRL - DEBUG - global iter =    8 total obj: 7793.6799
2021-07-02 14:28:24,253 - DeepRL - DEBUG - global iter =    9 total obj: 7256.8129
2021-07-02 14:28:42,705 - DeepRL - DEBUG - global iter =   10 total obj: 6667.4607
2021-07-02 14:29:01,879 - DeepRL - DEBUG - global iter =   11 total obj: 6443.5515


INFO:tensorflow:../results/simpleModel_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[4, 3, 3, ..., 4, 4, 3],
          [2, 1, 1, ..., 2, 2, 1],
          [4, 3, 4, ..., 3, 4, 4],
          ...,
          [2, 1, 1, ..., 2, 2, 2],
          [4, 3, 4, ..., 3, 4, 4],
          [2, 1, 2, ..., 1, 2, 2]], dtype=int32),
   'action': array([[0, 0, 1, ..., 1, 0, 1],
          [0, 0, 1, ..., 1, 0, 1],
          [0, 1, 1, ..., 0, 1, 1],
          ...,
          [0, 0, 1, ..., 0, 1, 0],
          [0, 1, 1, ..., 0, 1, 1],
          [0, 1, 1, ..., 0, 1, 1]]),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [3, 1]],
   
          [[1, 2],
           [3, 1],
           [1, 3],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          ...,
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ..

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

and we want to test the model every 10 iterations:

In [7]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-07-02 14:45:33,233 - DeepRL - DEBUG - version control: (None, None)
2021-07-02 14:45:33,235 - DeepRL - DEBUG - learning rate: 0.01
2021-07-02 14:45:33,236 - DeepRL - DEBUG - global iters: 50
2021-07-02 14:45:33,237 - DeepRL - DEBUG - training data-points: 90
2021-07-02 14:45:33,238 - DeepRL - DEBUG - test data-points: 10
2021-07-02 14:45:54,905 - DeepRL - DEBUG - opt started...
2021-07-02 14:45:54,908 - DeepRL - DEBUG - started testing...
2021-07-02 14:47:58,206 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_rewardOutcome/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:48:35,081 - DeepRL - DEBUG - global iter =    0 total obj: 11918.8022
2021-07-02 14:48:53,081 - DeepRL - DEBUG - global iter =    1 total obj: 11676.9832
2021-07-02 14:49:12,260 - DeepRL - DEBUG - global iter =    2 total obj: 11455.0541
2021-07-02 14:49:29,745 - DeepRL - DEBUG - global iter =    3 total obj: 10944.7837
2021-07-02 14:49:47,204 - DeepRL - DEBUG - global iter =    4 total obj: 10602.9990
2021-07-02 14:50:05,070 - DeepRL - DEBUG - global iter =    5 total obj: 10142.4682
2021-07-02 14:50:22,666 - DeepRL - DEBUG - global iter =    6 total obj: 9652.6956
2021-07-02 14:50:37,609 - DeepRL - DEBUG - global iter =    7 total obj: 8946.5448
2021-07-02 14:50:54,300 - DeepRL - DEBUG - global iter =    8 total obj: 8244.0284
2021-07-02 14:51:08,659 - DeepRL - DEBUG - global iter =    9 total obj: 7703.1442
2021-07-02 14:51:08,661 - DeepRL - DEBUG - started testing...
2021-07-02 14:52:40,231 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_rewardOutcome/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 14:53:13,782 - DeepRL - DEBUG - global iter =   10 total obj: 7255.7047
2021-07-02 14:53:31,105 - DeepRL - DEBUG - global iter =   11 total obj: 6924.2353
2021-07-02 14:53:47,297 - DeepRL - DEBUG - global iter =   12 total obj: 6565.2304
2021-07-02 14:54:06,335 - DeepRL - DEBUG - global iter =   13 total obj: 6339.2490
2021-07-02 14:54:24,438 - DeepRL - DEBUG - global iter =   14 total obj: 5966.0984
2021-07-02 14:54:41,967 - DeepRL - DEBUG - global iter =   15 total obj: 5755.6930
2021-07-02 14:54:59,154 - DeepRL - DEBUG - global iter =   16 total obj: 5486.3814
2021-07-02 14:55:16,412 - DeepRL - DEBUG - global iter =   17 total obj: 5321.2268
2021-07-02 14:55:30,541 - DeepRL - DEBUG - global iter =   18 total obj: 5102.4711
2021-07-02 14:55:42,223 - DeepRL - DEBUG - global iter =   19 total obj: 5187.3087
2021-07-02 14:55:42,225 - DeepRL - DEBUG - started testing...
2021-07-02 14:57:39,382 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/simpleModel_rewardOutcome/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:03:22,082 - DeepRL - DEBUG - global iter =   30 total obj: 4826.5691
2021-07-02 15:03:36,315 - DeepRL - DEBUG - global iter =   31 total obj: 4344.7463
2021-07-02 15:03:47,835 - DeepRL - DEBUG - global iter =   32 total obj: 4349.3325
2021-07-02 15:03:59,275 - DeepRL - DEBUG - global iter =   33 total obj: 4128.5263
2021-07-02 15:04:11,246 - DeepRL - DEBUG - global iter =   34 total obj: 3970.4645
2021-07-02 15:04:23,571 - DeepRL - DEBUG - global iter =   35 total obj: 4324.7966
2021-07-02 15:04:35,232 - DeepRL - DEBUG - global iter =   36 total obj: 4130.5231
2021-07-02 15:04:46,918 - DeepRL - DEBUG - global iter =   37 total obj: 4217.4543
2021-07-02 15:04:57,370 - DeepRL - DEBUG - global iter =   38 total obj: 4273.9024
2021-07-02 15:05:07,667 - DeepRL - DEBUG - global iter =   39 total obj: 3964.8051
2021-07-02 15:05:07,669 - DeepRL - DEBUG - started testing...
2021-07-02 15:07:45,933 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/simpleModel_rewardOutcome/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-07-02 15:08:02,951 - DeepRL - DEBUG - global iter =   40 total obj: 3746.4229
2021-07-02 15:08:14,218 - DeepRL - DEBUG - global iter =   41 total obj: 3737.4180
2021-07-02 15:08:25,011 - DeepRL - DEBUG - global iter =   42 total obj: 3617.2319
2021-07-02 15:08:34,674 - DeepRL - DEBUG - global iter =   43 total obj: 3937.1662
2021-07-02 15:08:45,842 - DeepRL - DEBUG - global iter =   44 total obj: 3813.0780
2021-07-02 15:08:57,003 - DeepRL - DEBUG - global iter =   45 total obj: 3592.8833
2021-07-02 15:09:08,258 - DeepRL - DEBUG - global iter =   46 total obj: 3681.7795
2021-07-02 15:09:19,662 - DeepRL - DEBUG - global iter =   47 total obj: 3630.2513
2021-07-02 15:09:30,873 - DeepRL - DEBUG - global iter =   48 total obj: 4689.7582
2021-07-02 15:09:44,593 - DeepRL - DEBUG - global iter =   49 total obj: 3759.5083
2021-07-02 15:09:44,595 - DeepRL - DEBUG - opt finished.
2021-07-02 15:09:44,596 - DeepRL - DEBUG - started testing...
2021-07-02 15:12:00,472 - DeepRL - DEBUG - finished

INFO:tensorflow:../results/simpleModel_rewardOutcome/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
test_data["S100"]

[{'action': array([[0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
          0, 0, 0, 1]]),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [2, 1],
          [1, 3],
          [2, 1],
          [2, 1],
          [1, 3],
          [3, 2],
          [3, 1],
          [1, 3],
          [2, 3],
          [1, 2],
          [1, 3],
          [2, 1],
          [1, 3],
          [3, 1],
          [1, 2],
          [2, 3],
          [2, 1],
          [3, 1],
          [2, 1],
          [3, 2],
          [1, 2],
          [1, 2],
          [2, 3],
          [1, 3],
          [3, 2],
          [2, 3],
          [3, 1],
          [3, 1],
          [2